In [2]:
!pip install dask

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dask.dataframe as dd
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import zipfile
import pandas as pd

train = pd.read_csv('../data/quora-question-pairs/train.csv.zip').dropna()
test = pd.read_csv('../data/quora-question-pairs/test.csv').dropna()


In [3]:
# collect all the words in the training data

# initialize dataframe with question and qid
qid1 = train['qid1'].values.tolist()
qid2 = train['qid2'].values.tolist()
question1 = train['question1'].values.tolist()
question2 = train['question2'].values.tolist()
qid_quest1 = pd.DataFrame.from_dict({'qid':qid1, 'question':question1})
qid_quest2 = pd.DataFrame.from_dict({'qid':qid2, 'question':question2})
qid_quest = pd.concat([qid_quest1,qid_quest2])

# collect all questions and count vectorize them
questions = qid_quest['question'].values.tolist()

In [4]:
qid_quest.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 808574 entries, 0 to 404286
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   qid       808574 non-null  int64 
 1   question  808574 non-null  object
dtypes: int64(1), object(1)
memory usage: 18.5+ MB


In [5]:
import nltk
from nltk.corpus import stopwords,wordnet
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
nltk.download("punkt")
nltk.download('averaged_perceptron_tagger')

stop_words = set(stopwords.words('english')) 
lemmatizer = WordNetLemmatizer() # Word net lemmatizer is pretty elaborate in its approach read this : https://www.machinelearningplus.com/nlp/lemmatization-examples-python/
    
def remove_stopwords (sentence, stopwords_list):
    """This method removes stop words from an untokenized/tokenized sentence and returns a token of strings"""
    final_tokens = []
    for each_word in sentence:
        if each_word not in stopwords_list:
            final_tokens.append(each_word)
    return final_tokens

def get_part_of_speech(sentence):
    """This method gets the part of speech in context of the sentence the word is in, 
    this is better than using the vanilla lemmatizer without any part of speech tag. 
    Also this returns Wordnet tags which can directly
    be used in the wordnet lemmatizer. Use this before removing stop words. Note this is NLTK specific"""
    tagged_sentence = nltk.pos_tag(nltk.word_tokenize(sentence))
    return tagged_sentence

def convert_pos_to_wordnet(word, nltk_tag):
    """This method converts nltk tag to wordnet tag"""
    wn_pos = None
    if nltk_tag.startswith('J'):
        wn_pos = wordnet.ADJ
    elif nltk_tag.startswith('V'):
        wn_pos = wordnet.VERB
    elif nltk_tag.startswith('N'):
        wn_pos = wordnet.NOUN
    elif nltk_tag.startswith('R'):
        wn_pos = wordnet.ADV
    else:          
        wn_pos = wordnet.NOUN
    return word, wn_pos

    
def lemmatize_sentence(sentence, lemmatizer=lemmatizer):
    """This method uses an nltk lemmatizer on a sentence"""
    wn_tagged_sent = [lemmatizer.lemmatize(convert_pos_to_wordnet(x[0],x[1])[0], pos=convert_pos_to_wordnet(x[0],x[1])[1]).lower() for x in get_part_of_speech(sentence)  if not x[0] in stop_words and x[0].isalnum()]
    return wn_tagged_sent

# multiprocessing ready lemmatize
def lemmatize_sentence_mp(sentence):
    """This method uses an nltk lemmatizer on a sentence"""
    wn_tagged_sent = [convert_pos_to_wordnet(x[0],x[1]) for x in get_part_of_speech(sentence)]
    return " ".join([lemmatizer.lemmatize(x[0], pos=x[1]) for x in wn_tagged_sent if not x[0] in stop_words and x[0].isalnum()])

test_sent = questions[0]
print(test_sent, "\n",lemmatize_sentence(test_sent,lemmatizer))




[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/prabhatturlapati/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/prabhatturlapati/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/prabhatturlapati/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


What is the step by step guide to invest in share market in india? 
 ['what', 'step', 'step', 'guide', 'invest', 'share', 'market', 'india']


In [6]:
def count(sentence):
    return len(sentence.split())
max(qid_quest['question'].apply(count))

237

In [7]:
# res = qid_quest.apply(lambda x: lemmatize_sentence(x['question']), axis=1)

In [8]:
qid_quest

,qid,question
0,1,What is the step by step guide to invest in sh...
1,3,What is the story of Kohinoor (Koh-i-Noor) Dia...
2,5,How can I increase the speed of my internet co...
3,7,Why am I mentally very lonely? How can I solve...
4,9,"Which one dissolve in water quikly sugar, salt..."
...,...,...
404282,379845,How many keywords are there in PERL Programmin...
404283,155606,Is it true that there is life after death?
404284,537929,What's this coin?
404285,537931,I am having little hairfall problem but I want...


In [9]:
%%time
import multiprocessing as mp
import numpy as np
import pandas.util.testing as pdt

# def process_apply(x):
#     # do some stuff to data here

def process(df):
    res = df.apply(lambda x: lemmatize_sentence(x['question']), axis=1)
    return res

print(4*mp.cpu_count())
p = mp.Pool(processes=4*mp.cpu_count())
split_dfs = np.array_split(qid_quest,4*mp.cpu_count())
pool_results = p.map(process, split_dfs)
p.close()
p.join()
    
    
# merging parts processed by different processes
parts = pd.concat(pool_results, axis=0)

# merging newly calculated parts to big_df
qid_quest = pd.concat([qid_quest, parts], axis=1)

# # checking if the dfs were merged correctly
# pdt.assert_series_equal(parts['id'], big_df['id'])

32


/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  This is separate from the ipykernel package so we can avoid doing imports until
Process ForkPoolWorker-25:
Process ForkPoolWorker-21:
Process ForkPoolWorker-13:
Process ForkPoolWorker-5:
Process ForkPoolWorker-30:
Process ForkPoolWorker-4:
Process ForkPoolWorker-29:
Process ForkPoolWorker-24:
Process ForkPoolWorker-32:
Process ForkPoolWorker-26:
Process ForkPoolWorker-20:
Process ForkPoolWorker-11:
Process ForkPoolWorker-31:
Process ForkPoolWorker-8:
Process ForkPoolWorker-15:
Process ForkPoolWorker-22:
Process ForkPoolWorker-1:
Process ForkPoolWorker-7:
Process ForkPoolWorker-14:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call las

  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/site-packages/nltk/tag/perceptron.py", line 245, in load
    self.model.weights, self.tagdict, self.classes = load(loc)
  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    r

  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py", line 871, in __getitem__
    result = self.index.get_value(self, key)
  File "<ipython-input-5-47acf73d3ce4>", line 24, in get_part_of_speech
    tagged_sentence = nltk.pos_tag(nltk.word_tokenize(sentence))
  File "<ipython-input-5-47acf73d3ce4>", line 24, in get_part_of_speech
    tagged_sentence = nltk.pos_tag(nltk.word_tokenize(sentence))
  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py", line 871, in __getitem__
    result = self.index.get_value(self, key)
  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 4405, in get_value
    return self._engine.get_value(s, k, tz=getattr(series.dtype, "tz", None))
  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/site-packages/nltk/tag/__init__.py", line 162, in pos_tag
    return _pos_tag(tokens, tagset, tagger, lang)
  File "<ipython

  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Process ForkPoolWorker-18:
  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/prabhatturlapati/opt

...

  File "<timed exec>", line 9, in <lambda>
  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/site-packages/pandas/core/apply.py", line 296, in apply_standard
    values, self.f, axis=self.axis, dummy=dummy, labels=labels
  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/site-packages/pandas/core/apply.py", line 296, in apply_standard
    values, self.f, axis=self.axis, dummy=dummy, labels=labels
  File "pandas/_libs/reduction.pyx", line 618, in pandas._libs.reduction.compute_reduction
  File "<timed exec>", line 9, in <lambda>
  File "pandas/_libs/reduction.pyx", line 618, in pandas._libs.reduction.compute_reduction
  File "<timed exec>", line 9, in <lambda>
  File "pandas/_libs/reduction.pyx", line 128, in pandas._libs.reduction.Reducer.get_result
  File "pandas/_libs/reduction.pyx", line 618, in pandas._libs.reduction.compute_reduction
  File "<timed exec>", line 9, in <lambda>
  File "<timed exec>", line 9, in <lambda>
  File "<timed exec>", line 9, in <lambda>
 

  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py", line 869, in __getitem__
    key = com.apply_if_callable(key, self)
  File "<ipython-input-5-47acf73d3ce4>", line 24, in get_part_of_speech
    tagged_sentence = nltk.pos_tag(nltk.word_tokenize(sentence))
  File "<ipython-input-5-47acf73d3ce4>", line 45, in lemmatize_sentence
    wn_tagged_sent = [lemmatizer.lemmatize(convert_pos_to_wordnet(x[0],x[1])[0], pos=convert_pos_to_wordnet(x[0],x[1])[1]).lower() for x in get_part_of_speech(sentence)  if not x[0] in stop_words and x[0].isalnum()]
  File "<ipython-input-5-47acf73d3ce4>", line 24, in get_part_of_speech
    tagged_sentence = nltk.pos_tag(nltk.word_tokenize(sentence))
  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/site-packages/nltk/tag/__init__.py", line 162, in pos_tag
    return _pos_tag(tokens, tagset, tagger, lang)
  File "<ipython-input-5-47acf73d3ce4>", line 24, in get_part_of_speech
    tagged_sentence = nltk.pos_t

  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/site-packages/nltk/tag/perceptron.py", line 180, in tag
    tag, conf = self.model.predict(features, return_conf)
  File "<ipython-input-5-47acf73d3ce4>", line 24, in get_part_of_speech
    tagged_sentence = nltk.pos_tag(nltk.word_tokenize(sentence))
  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/site-packages/nltk/tag/perceptron.py", line 179, in tag
    features = self._get_features(i, word, context, prev, prev2)
  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/site-packages/nltk/tag/__init__.py", line 119, in _pos_tag
    tagged_tokens = tagger.tag(tokens)
  File "<ipython-input-5-47acf73d3ce4>", line 45, in lemmatize_sentence
    wn_tagged_sent = [lemmatizer.lemmatize(convert_pos_to_wordnet(x[0],x[1])[0], pos=convert_pos_to_wordnet(x[0],x[1])[1]).lower() for x in get_part_of_speech(sentence)  if not x[0] in stop_words and x[0].isalnum()]
  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/si

  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/site-packages/nltk/tag/perceptron.py", line 69, in predict
    scores[label] += value * weight
  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/site-packages/nltk/tag/__init__.py", line 162, in pos_tag
    return _pos_tag(tokens, tagset, tagger, lang)
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/site-packages/nltk/tag/__init__.py", line 162, in pos_tag
    return _pos_tag(tokens, tagset, tagger, lang)
  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/site-packages/nltk/tag/perceptron.py", line 162, in __init__
    find('taggers/averaged_perceptron_tagger/' + PICKLE)
KeyboardInterrupt
  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/site-packages/nltk/tokenize/treebank.py", line 122, in tokenize
    text = regexp.sub(substitution, text)
  File "/Users/prabhatturlapati/opt/anaconda3/lib/python3.7/site-packages/nltk/data.py", line

KeyboardInterrupt: 

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = qid_quest['question'].tolist()
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
print(X.shape)

['00', '000', '0000', '000000', '00000000', '0000000000', '0000001', '00000074', '0000021210', '00001', '0001', '0002', '000ft', '000inr', '000k', '000rs', '000s', '000usd', '000webhost', '001', '0015', '0019', '002', '0021', '0035', '003sc', '005', '0051', '007', '008', '008801703772104', '0088882578743', '00am', '00g', '00ghz', '00pm', '00s', '00v', '01', '0113', '012', '013j', '014', '018v4', '01gbp', '01st', '02', '020', '02038', '022', '025', '026', '03', '032', '0320', '034', '035', '035tx', '036', '038', '03g', '04', '04009', '041x', '042', '045', '04lts', '05', '050', '052', '0524', '05246', '054', '055', '0569', '058', '05_mar', '06', '062', '065g', '066', '0665745511651039', '067972477x', '068', '06x08', '07', '0706', '072', '0745', '075', '07904378172', '079tx', '07ish', '08', '080', '083', '086', '088', '08888888888', '088989914321', '08gb', '09', '090', '099', '09987510597', '0a', '0and', '0are', '0dev', '0f', '0g', '0gb', '0ghz', '0kg', '0l', '0m', '0mq', '0n7', '0s', '0v

In [11]:
tfidf_features = pd.DataFrame(X.todense(), columns=[vectorizer.get_feature_names()])
tfidf_features.head()

,00,000,0000,000000,00000000,0000000000,0000001,00000074,0000021210,00001,...,분위기,불타오르네,슬마,심하잖아,이정현,친구해도,쾌지나칭칭나네,하지만,한글,ﬁnd
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# tfidf_qid_quest = pd.concat([qid_quest,tfidf_features], axis=1)
tfidf_features.index = qid_quest['qid'].copy()

In [ ]:
tfidf_features = tfidf_features.sort_index()


In [ ]:
tfidf_features.head()

In [16]:
# distribution of is_duplicate

print("len of dataset : ", len(train))
print("number of duplicates : ", len(train.loc[train['is_duplicate']==1]), "ratio in dataset : ", len(train.loc[train['is_duplicate']==1])/len(train))
print("number of non-duplicates : ", len(train.loc[train['is_duplicate']==0]), "ratio in dataset : ", len(train.loc[train['is_duplicate']==0])/len(train))


len of dataset :  404287
number of duplicates :  149263 ratio in dataset :  0.3692005926482919
number of non-duplicates :  255024 ratio in dataset :  0.630799407351708


In [14]:
#  Doc2Vec Approach
len(parts)

808574

In [ ]:
# Vectorize Q1 and Q2 for each row
import gensim
import gensim.downloader as api
from gensim import corpora
from gensim.matutils import softcossim

sent_1 = 'Dravid is a cricket player and a opening batsman'.split()
sent_2 = 'Leo is a cricket player too He is a batsman,baller and keeper'.split()

# Download the FastText model
fasttext_model300 = api.load('glove-twitter-25')




In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# apply function
def soft_cosine(sent_1, sent_2):
    sent_1 = sent_1.split()
    sent_2 = sent_2.split()
    dictionary = corpora.Dictionary([sent_1, sent_2])
    similarity_matrix = fasttext_model300.similarity_matrix(dictionary)
    t_sent_1 = dictionary.doc2bow(sent_1)
    t_sent_2 = dictionary.doc2bow(sent_2)
    sc = softcossim(t_sent_1, t_sent_2, similarity_matrix)
    return sc
    
def cosine(sent_1, sent_2):
#     sent_1 = sent_1.split()
#     sent_2 = sent_2.split()
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform([sent_1, sent_2])
    cs = cosine_similarity(X[0],X[1])
    return cs[0][0]
#     dictionary = corpora.Dictionary([sent_1, sent_2])
#     similarity_matrix = fasttext_model300.similarity_matrix(dictionary)
#     t_sent_1 = dictionary.doc2bow(sent_1)
#     t_sent_2 = dictionary.doc2bow(sent_2)
#     sc = softcossim(t_sent_1, t_sent_2, similarity_matrix)
#     return sc
cosine(train.iloc[0]['question1'], train.iloc[0]['question2'])
    
# train['soft_cosine']= train.apply(lambda x: soft_cosine(x['question1'], x['question2']), axis=1)

In [ ]:
import multiprocessing
multiprocessing.cpu_count()

In [ ]:
sample_train = train.sample(n=10000)

In [ ]:
print("len of dataset : ", len(sample_train))
print("number of duplicates : ", len(sample_train.loc[sample_train['is_duplicate']==1]), "ratio in dataset : ", len(train.loc[train['is_duplicate']==1])/len(train))
print("number of non-duplicates : ", len(sample_train.loc[sample_train['is_duplicate']==0]), "ratio in dataset : ", len(train.loc[train['is_duplicate']==0])/len(train))


In [ ]:
from multiprocessing import Pool
from functools import partial
import numpy as np

def parallelize(data, func, num_of_processes=8):
    data_split = np.array_split(data, num_of_processes)
    pool = Pool(num_of_processes)
    data = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return data

def run_on_subset(data_subset):
    return data_subset.apply(lambda x: soft_cosine(x['question1'], x['question2']), axis=1)
#     return data_subset.apply(lambda x: cosine(x['question1'], x['question2']), axis=1)

def parallelize_on_rows(data, num_of_processes=8):
    return parallelize(data, partial(run_on_subset), num_of_processes)


df_res = parallelize_on_rows(sample_train,8)

In [ ]:
sample_train['soft_cos'] = df_res


In [ ]:
import matplotlib.pyplot as plt

x = sample_train.loc[sample_train['is_duplicate']==1]['soft_cos']
y = sample_train.loc[sample_train['is_duplicate']==0]['soft_cos']
plt.hist([x, y], 10, label=['1', '0'])
plt.legend(loc='upper right')
plt.show()

print("Average Similarity for duplicates : ", np.mean(x))
print("Average Similarity for not duplicates : ", np.mean(y))

In [ ]:
x = sample_train.loc[sample_train['is_duplicate']==1]['soft_cos']
y = sample_train.loc[sample_train['is_duplicate']==0]['soft_cos']
plt.boxplot([x, y])
plt.legend(loc='upper right')
plt.show()



In [ ]:
sample_train.loc[sample_train['is_duplicate']==0]
threshold = 0.862
sample_train['predicted_is_duplicate'] = np.where(sample_train['soft_cos']>threshold,1,0)

In [ ]:
from sklearn.metrics import accuracy_score

print("Accuracy ",accuracy_score(sample_train['is_duplicate'],sample_train['predicted_is_duplicate']))

In [ ]:
# Using above threshold , test the test data
test_res= parallelize_on_rows(test,8)